<a href="https://colab.research.google.com/github/lmbd92/DataScienceMonograph/blob/main/Notebooks/iteracion1ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset retail ventas online minoristas y mayoristas

In [1]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np


# Matemáticas y estadísticas
# ==============================================================================
import math

# Preparación de datos
# ==============================================================================
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
import time
from sklearn.metrics import mean_squared_error, r2_score
from pylab import *
from pandas import DataFrame
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_validate
from statistics import mean
from sklearn.linear_model import LinearRegression



# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')



## 1. Carga de datos

In [2]:
url = 'https://raw.githubusercontent.com/lmbd92/DataScienceMonograph/main/Data/data-raw/online_retail_II.csv'
df = pd.read_csv(url)
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [4]:
df.shape

(1067371, 8)

In [5]:
df.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

In [6]:
df.nunique()

Invoice        53628
StockCode       5305
Description     5698
Quantity        1057
InvoiceDate    47635
Price           2807
Customer ID     5942
Country           43
dtype: int64

In [7]:
df.describe()

,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,824364.000000
mean,9.938898e+00,4.649388e+00,15324.638504
std,1.727058e+02,1.235531e+02,1697.464450
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13975.000000
50%,3.000000e+00,2.100000e+00,15255.000000
75%,1.000000e+01,4.150000e+00,16797.000000
max,8.099500e+04,3.897000e+04,18287.000000


## 2. Limpieza de datos

In [8]:
#Borrando columnas que no se emplearán

df.drop([ 'Customer ID', 'Description'], axis='columns', inplace=True)


In [9]:
# Definimos una lista de columnas categoricas y otra de columnas numericas

catCols = df.select_dtypes(include = ["object", 'category']).columns.tolist()
numCols=df.select_dtypes(include = ['float64','float64','int32','int64']).columns.tolist()
catCols

['Invoice', 'StockCode', 'InvoiceDate', 'Country']

In [10]:
numCols

['Quantity', 'Price']

In [11]:
# Eliminar los caracteres no numéricos de la variable "Invoice"

df['Invoice'] = df['Invoice'].str.replace(r'\D', '', regex=True)


In [12]:
# Limpieza de las variables "Quantity" y "Price" ya que presentan valores negativos correspondientes a casos de devoluciones
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

#Reiniciar el indice
df.reset_index(drop=True, inplace=True)

In [13]:
# Calculo de valores atípicos para la variable "Quantity"


#Calculo de Q1 t Q3
Q1 = np.percentile(df['Quantity'], 25, interpolation = 'midpoint')
Q3 = np.percentile(df['Quantity'], 75, interpolation = 'midpoint')

#Cálculo del rango intercuartil
IQR = Q3 - Q1

#Cálculo de valor mínimo y máximo para los valores atípicos
VAInf = Q1 - 10*IQR
VASup = Q3 + 10*IQR

print(f'Valor atípico leve inferior:{VAInf}')
print(f'Valor atípico leve superior:{VASup}')

Valor atípico leve inferior:-89.0
Valor atípico leve superior:100.0


In [14]:
# Se eliminan los valores atípicos
df = df.drop(df[df['Quantity']>VASup].index)

#Reiniciar el indice
df.reset_index(drop=True, inplace=True)

In [15]:
df.shape

(1031377, 6)

## 3. Preparación de datos

In [16]:
# Generación de nuevas variables
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df['Dates'] = pd.to_datetime(df['InvoiceDate']).dt.date
df['Year'] = pd.to_datetime(df['InvoiceDate']).dt.year
df['Months'] = pd.to_datetime(df['InvoiceDate']).dt.month
df['Month_day'] = pd.to_datetime(df['InvoiceDate']).dt.day
df['Time_hour'] = pd.to_datetime(df['InvoiceDate']).dt.hour
df['wk_day'] = df['InvoiceDate'].dt.dayofweek + 1
df['year_month'] = df['InvoiceDate'].dt.strftime('%Y.%m')
df['TotalSpent'] = df['Quantity']*df['Price']
df.shape

(1031377, 14)

In [17]:
# Calculo de total por transacción "Invoice" y cantidad total de items por transacción

df['TotalTransaction'] = df.groupby('Invoice')['TotalSpent'].transform('sum')
df['TotalQuantity'] = df.groupby('Invoice')['Quantity'].transform('sum')
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Country,Dates,Year,Months,Month_day,Time_hour,wk_day,year_month,TotalSpent,TotalTransaction,TotalQuantity
0,489434,85048,12,2009-12-01 07:45:00,6.95,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,83.4,505.3,166
1,489434,79323P,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166
2,489434,79323W,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166
3,489434,22041,48,2009-12-01 07:45:00,2.10,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,100.8,505.3,166
4,489434,21232,24,2009-12-01 07:45:00,1.25,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,30.0,505.3,166


In [18]:
# Calculo de productos  únicos por transacción

df['TotalProductosUnicos'] = df.groupby('Invoice')['StockCode'].transform('nunique')
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Country,Dates,Year,Months,Month_day,Time_hour,wk_day,year_month,TotalSpent,TotalTransaction,TotalQuantity,TotalProductosUnicos
0,489434,85048,12,2009-12-01 07:45:00,6.95,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,83.4,505.3,166,8
1,489434,79323P,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166,8
2,489434,79323W,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166,8
3,489434,22041,48,2009-12-01 07:45:00,2.10,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,100.8,505.3,166,8
4,489434,21232,24,2009-12-01 07:45:00,1.25,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,30.0,505.3,166,8


In [ ]:
!pip install pycountry_convert

In [20]:
# Función para agrupar por continente los paises

import pycountry_convert as pc

def obtener_continente(country):
    try:
        codigo_pais = pc.country_name_to_country_alpha2(country)
        codigo_continente = pc.country_alpha2_to_continent_code(codigo_pais)
        continente = pc.convert_continent_code_to_continent_name(codigo_continente)
        return continente
    except KeyError:
        return "unknown"

df['Continent'] = df['Country'].apply(obtener_continente)

In [21]:
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Country,Dates,Year,Months,Month_day,Time_hour,wk_day,year_month,TotalSpent,TotalTransaction,TotalQuantity,TotalProductosUnicos,Continent
0,489434,85048,12,2009-12-01 07:45:00,6.95,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,83.4,505.3,166,8,Europe
1,489434,79323P,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166,8,Europe
2,489434,79323W,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166,8,Europe
3,489434,22041,48,2009-12-01 07:45:00,2.10,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,100.8,505.3,166,8,Europe
4,489434,21232,24,2009-12-01 07:45:00,1.25,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,30.0,505.3,166,8,Europe


In [22]:
# Visualizar los valores unicos de la nueva variable
df["Continent"].unique()

array(['Europe', 'North America', 'Oceania', 'unknown', 'Asia', 'Africa',
       'South America'], dtype=object)

In [23]:
# Visualizar los registros con etiqueta "unknown"

df_resultado = df[df["Continent"] == "unknown"]
df_resultado

,Invoice,StockCode,Quantity,InvoiceDate,Price,Country,Dates,Year,Months,Month_day,Time_hour,wk_day,year_month,TotalSpent,TotalTransaction,TotalQuantity,TotalProductosUnicos,Continent
405,489520,35751C,12,2009-12-01 11:41:00,0.75,EIRE,2009-12-01,2009,12,1,11,2,2009.12,9.0,733.63,293,30,unknown
406,489520,72739B,12,2009-12-01 11:41:00,1.25,EIRE,2009-12-01,2009,12,1,11,2,2009.12,15.0,733.63,293,30,unknown
407,489520,22114,8,2009-12-01 11:41:00,3.95,EIRE,2009-12-01,2009,12,1,11,2,2009.12,31.6,733.63,293,30,unknown
408,489520,22212,6,2009-12-01 11:41:00,2.10,EIRE,2009-12-01,2009,12,1,11,2,2009.12,12.6,733.63,293,30,unknown
409,489520,21523,2,2009-12-01 11:41:00,6.75,EIRE,2009-12-01,2009,12,1,11,2,2009.12,13.5,733.63,293,30,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028666,581433,22192,2,2011-12-08 15:54:00,8.50,EIRE,2011-12-08,2011,12,8,15,4,2011.12,17.0,1084.14,806,72,unknown
1028667,581433,48187,2,2011-12-08 15:54:00,8.25,EIRE,2011-12-08,2011,12,8,15,4,2011.12,16.5,1084.14,806,72,unknown
1028668,581433,48184,2,2011-12-08 15:54:00,8.25,EIRE,2011-12-08,2011,12,8,15,4,2011.12,16.5,1084.14,806,72,unknown
1028669,581433,20685,2,2011-12-08 15:54:00,8.25,EIRE,2011-12-08,2011,12,8,15,4,2011.12,16.5,1084.14,806,72,unknown


In [24]:
# Visualizar los paises que la función no pudo clasificar
df_resultado["Country"].unique()

array(['EIRE', 'Channel Islands', 'Unspecified', 'RSA', 'West Indies',
       'Korea', 'European Community'], dtype=object)

In [25]:
# Eliminar los registros del DF original con Country = "Unspecified"

df = df[df["Country"] != "Unspecified"]
df.shape

(1030626, 18)

In [26]:
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Country,Dates,Year,Months,Month_day,Time_hour,wk_day,year_month,TotalSpent,TotalTransaction,TotalQuantity,TotalProductosUnicos,Continent
0,489434,85048,12,2009-12-01 07:45:00,6.95,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,83.4,505.3,166,8,Europe
1,489434,79323P,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166,8,Europe
2,489434,79323W,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166,8,Europe
3,489434,22041,48,2009-12-01 07:45:00,2.10,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,100.8,505.3,166,8,Europe
4,489434,21232,24,2009-12-01 07:45:00,1.25,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,30.0,505.3,166,8,Europe


In [27]:
# Etiquetar manualmente los demás paises

df.loc[df['Country'] == 'EIRE', 'Continent'] = 'Europe'
df.loc[df['Country'] == 'Channel Islands', 'Continent'] = 'Europe'
df.loc[df['Country'] == 'RSA', 'Continent'] = 'Africa'
df.loc[df['Country'] == 'West Indies', 'Continent'] = 'North_America'
df.loc[df['Country'] == 'Korea', 'Continent'] = 'Asia'
df.loc[df['Country'] == 'European Community', 'Continent'] = 'Europe'

In [28]:
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Country,Dates,Year,Months,Month_day,Time_hour,wk_day,year_month,TotalSpent,TotalTransaction,TotalQuantity,TotalProductosUnicos,Continent
0,489434,85048,12,2009-12-01 07:45:00,6.95,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,83.4,505.3,166,8,Europe
1,489434,79323P,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166,8,Europe
2,489434,79323W,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.0,505.3,166,8,Europe
3,489434,22041,48,2009-12-01 07:45:00,2.10,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,100.8,505.3,166,8,Europe
4,489434,21232,24,2009-12-01 07:45:00,1.25,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,30.0,505.3,166,8,Europe


In [29]:
print(df["Continent"].value_counts())

Europe           1024674
Asia                3427
Oceania             1515
North America        665
Africa               197
South America         94
North_America         54
Name: Continent, dtype: int64


In [30]:
# Estandarizando la variable "StockCode" y "Country" ya que es de tipo objeto y se evidencian caracteres en su composición

df['CodigoUnico'] = pd.factorize(df['StockCode'])[0] + 1
df['CountryUnico'] = pd.factorize(df['Country'])[0] + 1
df

,Invoice,StockCode,Quantity,InvoiceDate,Price,Country,Dates,Year,Months,Month_day,Time_hour,wk_day,year_month,TotalSpent,TotalTransaction,TotalQuantity,TotalProductosUnicos,Continent,CodigoUnico,CountryUnico
0,489434,85048,12,2009-12-01 07:45:00,6.95,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,83.40,505.30,166,8,Europe,1,1
1,489434,79323P,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.00,505.30,166,8,Europe,2,1
2,489434,79323W,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,81.00,505.30,166,8,Europe,3,1
3,489434,22041,48,2009-12-01 07:45:00,2.10,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,100.80,505.30,166,8,Europe,4,1
4,489434,21232,24,2009-12-01 07:45:00,1.25,United Kingdom,2009-12-01,2009,12,1,7,2,2009.12,30.00,505.30,166,8,Europe,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031372,581587,22899,6,2011-12-09 12:50:00,2.10,France,2011-12-09,2011,12,9,12,5,2011.12,12.60,267.45,106,16,Europe,4046,2
1031373,581587,23254,4,2011-12-09 12:50:00,4.15,France,2011-12-09,2011,12,9,12,5,2011.12,16.60,267.45,106,16,Europe,4324,2
1031374,581587,23255,4,2011-12-09 12:50:00,4.15,France,2011-12-09,2011,12,9,12,5,2011.12,16.60,267.45,106,16,Europe,4326,2
1031375,581587,22138,3,2011-12-09 12:50:00,4.95,France,2011-12-09,2011,12,9,12,5,2011.12,14.85,267.45,106,16,Europe,72,2


In [31]:
# Verificamos que los valores unicos de las variables "StockCode" y "CodigoUnico" coincidan al igual que "Country" y "CountryUnico"
df.nunique()

Invoice                 39184
StockCode                4908
Quantity                  100
InvoiceDate             36431
Price                    2231
Country                    42
Dates                     604
Year                        3
Months                     12
Month_day                  31
Time_hour                  15
wk_day                      7
year_month                 25
TotalSpent               5809
TotalTransaction        28071
TotalQuantity            1459
TotalProductosUnicos      423
Continent                   7
CodigoUnico              4908
CountryUnico               42
dtype: int64

In [32]:
# Aplicación de la función de get_dummies a la variable "Continent"

df = pd.get_dummies(df, columns=['Continent'], drop_first=1)

In [33]:
df.shape
df

,Invoice,StockCode,Quantity,InvoiceDate,Price,Country,Dates,Year,Months,Month_day,...,TotalQuantity,TotalProductosUnicos,CodigoUnico,CountryUnico,Continent_Asia,Continent_Europe,Continent_North America,Continent_North_America,Continent_Oceania,Continent_South America
0,489434,85048,12,2009-12-01 07:45:00,6.95,United Kingdom,2009-12-01,2009,12,1,...,166,8,1,1,0,1,0,0,0,0
1,489434,79323P,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,...,166,8,2,1,0,1,0,0,0,0
2,489434,79323W,12,2009-12-01 07:45:00,6.75,United Kingdom,2009-12-01,2009,12,1,...,166,8,3,1,0,1,0,0,0,0
3,489434,22041,48,2009-12-01 07:45:00,2.10,United Kingdom,2009-12-01,2009,12,1,...,166,8,4,1,0,1,0,0,0,0
4,489434,21232,24,2009-12-01 07:45:00,1.25,United Kingdom,2009-12-01,2009,12,1,...,166,8,5,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031372,581587,22899,6,2011-12-09 12:50:00,2.10,France,2011-12-09,2011,12,9,...,106,16,4046,2,0,1,0,0,0,0
1031373,581587,23254,4,2011-12-09 12:50:00,4.15,France,2011-12-09,2011,12,9,...,106,16,4324,2,0,1,0,0,0,0
1031374,581587,23255,4,2011-12-09 12:50:00,4.15,France,2011-12-09,2011,12,9,...,106,16,4326,2,0,1,0,0,0,0
1031375,581587,22138,3,2011-12-09 12:50:00,4.95,France,2011-12-09,2011,12,9,...,106,16,72,2,0,1,0,0,0,0


In [34]:
# Eliminamos las columnas previamente codificadas
df = df.drop(columns=['StockCode','Country','InvoiceDate','Dates'])


In [35]:
df

,Invoice,Quantity,Price,Year,Months,Month_day,Time_hour,wk_day,year_month,TotalSpent,...,TotalQuantity,TotalProductosUnicos,CodigoUnico,CountryUnico,Continent_Asia,Continent_Europe,Continent_North America,Continent_North_America,Continent_Oceania,Continent_South America
0,489434,12,6.95,2009,12,1,7,2,2009.12,83.40,...,166,8,1,1,0,1,0,0,0,0
1,489434,12,6.75,2009,12,1,7,2,2009.12,81.00,...,166,8,2,1,0,1,0,0,0,0
2,489434,12,6.75,2009,12,1,7,2,2009.12,81.00,...,166,8,3,1,0,1,0,0,0,0
3,489434,48,2.10,2009,12,1,7,2,2009.12,100.80,...,166,8,4,1,0,1,0,0,0,0
4,489434,24,1.25,2009,12,1,7,2,2009.12,30.00,...,166,8,5,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031372,581587,6,2.10,2011,12,9,12,5,2011.12,12.60,...,106,16,4046,2,0,1,0,0,0,0
1031373,581587,4,4.15,2011,12,9,12,5,2011.12,16.60,...,106,16,4324,2,0,1,0,0,0,0
1031374,581587,4,4.15,2011,12,9,12,5,2011.12,16.60,...,106,16,4326,2,0,1,0,0,0,0
1031375,581587,3,4.95,2011,12,9,12,5,2011.12,14.85,...,106,16,72,2,0,1,0,0,0,0


Salvando el DF con datos limpios y transformados, previo a la implementación de los algoritmos ML

In [36]:
from google.colab import files

df.to_csv('online_retail_II_limpio.csv', index=False)

files.download('online_retail_II_limpio.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 4. Machine Learning

### 4.1 Regresión Lineal Simple

In [37]:
# Monitoreo del tiempo de procesamiento
import time

start_time = time.time()

print('-------------------INICIO PROCESAMIENTO-----------------')

-------------------INICIO PROCESAMIENTO-----------------


In [38]:
# Definición variables de entrada y variable de salida

df1= df.drop(columns=['TotalTransaction'])

df2 = df[['TotalTransaction']].copy()


In [39]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030626 entries, 0 to 1031376
Data columns (total 20 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Invoice                  1030626 non-null  object 
 1   Quantity                 1030626 non-null  int64  
 2   Price                    1030626 non-null  float64
 3   Year                     1030626 non-null  int64  
 4   Months                   1030626 non-null  int64  
 5   Month_day                1030626 non-null  int64  
 6   Time_hour                1030626 non-null  int64  
 7   wk_day                   1030626 non-null  int64  
 8   year_month               1030626 non-null  object 
 9   TotalSpent               1030626 non-null  float64
 10  TotalQuantity            1030626 non-null  int64  
 11  TotalProductosUnicos     1030626 non-null  int64  
 12  CodigoUnico              1030626 non-null  int64  
 13  CountryUnico             1030626 non-null 

In [40]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030626 entries, 0 to 1031376
Data columns (total 1 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   TotalTransaction  1030626 non-null  float64
dtypes: float64(1)
memory usage: 15.7 MB


In [41]:
# Creación de variables para el test y el train

TiempoEntrenamientoTRAIN=[]
TiempoEvaluacionTRAIN=[]
RMSETrain=[]
R2Train=[]
MSETrain=[]
MAETrain=[]
nameColum=[]

TiempoEntrenamientoTEST=[]
TiempoEvaluacionTEST=[]
RMSETest=[]
R2Test=[]
MSETest=[]
MAETest=[]

In [42]:
Cantidad = 19
print('-------------------INICIO PROCESAMIENTO -----------------')

for i in range(Cantidad):

    #Selecciono predictores
    X = df1.iloc[:,i]

    #Selecciono target
    y = df2.iloc[:,0]

    NameVariable = df1.columns[i]
    nameColum.append(NameVariable)

    #separo datos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=4444)
    X_train = X_train.values.reshape(-1,1)
    X_test = X_test.values.reshape(-1,1)
    ## se escalan los datos
    escalar = MinMaxScaler(feature_range=(-1,1))
    X_train = escalar.fit_transform(X_train)
    X_test = escalar.fit_transform(X_test)
    #MODELO DE REGRESION LINEAL SIMPLE
    LR_simple = linear_model.LinearRegression(n_jobs=-1)
    scoring = ['r2','neg_root_mean_squared_error','neg_mean_squared_error','neg_mean_absolute_error']

    ###Con datos de Train
    scores = cross_validate(LR_simple, X_train, y_train, scoring=scoring, n_jobs=-1, cv=10)
    TiempoPromedioEntrenamientoTRAIN = mean(scores['fit_time'])
    TiempoPromedioEvaluacionTRAIN = mean(scores['score_time'])
    RMSEPromedioTRAIN = mean(scores['test_neg_root_mean_squared_error'])
    R2PromedioTRAIN = mean(scores['test_r2'])
    MSEPromedioTRAIN = mean(scores['test_neg_mean_squared_error'])
    MAEPromedioTRAIN = mean(scores['test_neg_mean_absolute_error'])
    TiempoEntrenamientoTRAIN.append(TiempoPromedioEntrenamientoTRAIN)
    TiempoEvaluacionTRAIN.append(TiempoPromedioEvaluacionTRAIN)
    RMSETrain.append(RMSEPromedioTRAIN)
    R2Train.append(R2PromedioTRAIN)
    MSETrain.append(MSEPromedioTRAIN)
    MAETrain.append(MAEPromedioTRAIN)

    RMSETrainFinal=pd.DataFrame(RMSETrain)
    R2TrainFinal=pd.DataFrame(R2Train)
    MSETrainFinal=pd.DataFrame(MSETrain)
    MAETrainFinal=pd.DataFrame(MAETrain)

    ###Con datos de Test
    scores = cross_validate(LR_simple, X_test, y_test, scoring=scoring, n_jobs=-1, cv=10)
    TiempoPromedioEntrenamientoTEST = mean(scores['fit_time'])
    TiempoPromedioEvaluacionTEST = mean(scores['score_time'])
    RMSEPromedioTEST = mean(scores['test_neg_root_mean_squared_error'])
    R2PromedioTEST = mean(scores['test_r2'])
    MSEPromedioTEST = mean(scores['test_neg_mean_squared_error'])
    MAEPromedioTEST = mean(scores['test_neg_mean_absolute_error'])
    TiempoEntrenamientoTEST.append(TiempoPromedioEntrenamientoTEST)
    TiempoEvaluacionTEST.append(TiempoPromedioEvaluacionTEST)
    RMSETest.append(RMSEPromedioTEST)
    R2Test.append(R2PromedioTEST)
    MSETest.append(MSEPromedioTEST)
    MAETest.append(MAEPromedioTEST)

    RMSETestFinal=pd.DataFrame(RMSETest)
    R2TestFinal=pd.DataFrame(R2Test)
    MSETestFinal=pd.DataFrame(MSETest)
    MAETestFinal=pd.DataFrame(MAETest)
    NameColumFinal=pd.DataFrame(nameColum)

RMSE1 = pd.concat([RMSETrainFinal,RMSETestFinal], axis=1)
R21 = pd.concat([R2TrainFinal,R2TestFinal], axis=1)
MSE1= pd.concat([MSETrainFinal,MSETestFinal], axis=1)
MAE1= pd.concat([MAETrainFinal,MAETestFinal], axis=1)
nameColum1= pd.concat([NameColumFinal], axis=1)

####Creacion Tablas
tabla1 = pd.concat([nameColum1,RMSE1,R21,MSE1,MAE1], axis=1)
tabla1.columns = ['Variable','RMSETrain','RMSETest','R2Train','R2Test','MSETrain','MSETest','MAETrain','MAETest']


print('--------------------------------------------------------------')
print('PROCESAMIENTO FINALIZADO EXITOSAMENTE!!!')
print('--------------------------------------------------------------')

TIEMPO = (time.time() - start_time)/60
print("--- %s Minutos ---" % (TIEMPO))

-------------------INICIO PROCESAMIENTO -----------------
--------------------------------------------------------------
PROCESAMIENTO FINALIZADO EXITOSAMENTE!!!
--------------------------------------------------------------
--- 0.5964657147725423 Minutos ---


In [43]:
tabla1

,Variable,RMSETrain,RMSETest,R2Train,R2Test,MSETrain,MSETest,MAETrain,MAETest
0,Invoice,-2107.370208,-2098.425127,0.005834,0.005841,-4.441607e+06,-4.404353e+06,-1189.517071,-1185.862949
1,Quantity,-2113.026162,-2104.051523,0.000490,0.000504,-4.465480e+06,-4.428005e+06,-1191.338150,-1187.725597
2,Price,-2112.358472,-2102.549003,0.001122,0.001919,-4.462660e+06,-4.421638e+06,-1191.069686,-1187.046354
3,Year,-2112.259421,-2103.243128,0.001215,0.001273,-4.462237e+06,-4.424608e+06,-1194.372826,-1190.701185
4,Months,-2070.677179,-2061.069696,0.040143,0.040909,-4.288242e+06,-4.248893e+06,-1205.027910,-1201.311327
5,Month_day,-2105.771293,-2096.783902,0.007338,0.007397,-4.434854e+06,-4.397469e+06,-1191.329686,-1187.704802
6,Time_hour,-2105.688270,-2096.676705,0.007422,0.007501,-4.434527e+06,-4.397033e+06,-1177.744641,-1174.014785
7,wk_day,-2095.407611,-2087.059389,0.017087,0.016570,-4.391317e+06,-4.356721e+06,-1168.338075,-1165.225222
8,year_month,-2113.008687,-2104.020999,0.000506,0.000534,-4.465405e+06,-4.427882e+06,-1193.609243,-1189.953471
9,TotalSpent,-2110.514542,-2100.320873,0.002864,0.004029,-4.454869e+06,-4.412255e+06,-1190.059944,-1185.924266


### 4.2 Regresión Lineal Multiple

In [44]:
# Monitoreo del tiempo de procesamiento
import time

start_time = time.time()

print('-------------------INICIO PROCESAMIENTO-----------------')

#Selecciono predictores
X = df1.to_numpy()


#Selecciono target
y = df2.iloc[:,0].to_numpy()

#separo datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=4444)

## se escalan los datos
escalar = MinMaxScaler(feature_range=(-1,1))
X_train = escalar.fit_transform(X_train)
X_test = escalar.fit_transform(X_test)

#MODELO DE REGRESION LINEAL MULTIPLE
LR_multiple = linear_model.LinearRegression(n_jobs=-1)

cv_results = cross_validate(LR_multiple, X_train, y_train, cv=10,n_jobs=-1,
              scoring=('neg_root_mean_squared_error', 'r2',
                       'neg_mean_squared_error', 'neg_mean_absolute_error'),
              return_train_score=True)

#Resultados Metricas
print('--------------------------------------------------------------')
print('Resultados RMSE')
print("RMSE_TRAIN: %0.3f" % (cv_results['train_neg_root_mean_squared_error'].mean()))
print("RMSE_TEST: %0.3f" % (cv_results['test_neg_root_mean_squared_error'].mean()))
print('--------------------------------------------------------------')

print('--------------------------------------------------------------')
print('Resultados R2')
print("R2_TRAIN: %0.3f" % (cv_results['train_r2'].mean()))
print("R2_TEST: %0.3f" % (cv_results['test_r2'].mean()))
print('--------------------------------------------------------------')

print('--------------------------------------------------------------')
print('Resultados MSE')
print("MSE_TRAIN: %0.3f" % (cv_results['train_neg_mean_squared_error'].mean()))
print("MSE_TEST: %0.3f" % (cv_results['test_neg_mean_squared_error'].mean()))
print('--------------------------------------------------------------')

print('--------------------------------------------------------------')
print('Resultados MAE')
print("MAE_TRAIN: %0.3f" % (cv_results['train_neg_mean_absolute_error'].mean()))
print("MAE_TEST: %0.3f" % (cv_results['test_neg_mean_absolute_error'].mean()))
print('--------------------------------------------------------------')

print('--------------------------------------------------------------')
print('PROCESAMIENTO FINALIZADO EXITOSAMENTE!!!')
print('--------------------------------------------------------------')

TIEMPO = (time.time() - start_time)/60
print("--- %s Minutos ---" % (TIEMPO))

-------------------INICIO PROCESAMIENTO-----------------
--------------------------------------------------------------
Resultados RMSE
RMSE_TRAIN: -818.619
RMSE_TEST: -850.561
--------------------------------------------------------------
--------------------------------------------------------------
Resultados R2
R2_TRAIN: 0.850
R2_TEST: 0.836
--------------------------------------------------------------
--------------------------------------------------------------
Resultados MSE
MSE_TRAIN: -670143.773
MSE_TEST: -733188.868
--------------------------------------------------------------
--------------------------------------------------------------
Resultados MAE
MAE_TRAIN: -433.151
MAE_TEST: -433.486
--------------------------------------------------------------
--------------------------------------------------------------
PROCESAMIENTO FINALIZADO EXITOSAMENTE!!!
--------------------------------------------------------------
--- 0.29501354694366455 Minutos ---


### 4.3 Regresión Lineal Multiple vs Regularización Ridge y Lasso

In [45]:
# Monitoreo del tiempo de procesamiento
import time

start_time = time.time()

print('-------------------INICIO PROCESAMIENTO-----------------')

# Seleccionar predictores y target
X = df1.values
y = df2.values

# Escalar los datos
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=4444)

# Modelo de Regresión Lineal Normal
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred_lm = lm.predict(X_test)
mse_lm = mean_squared_error(y_test, y_pred_lm)
r2_lm = r2_score(y_test, y_pred_lm)

# Modelo de Regresión Ridge
ridge = Ridge(alpha=0.5)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

# Modelo de Regresión Lasso
lasso = Lasso(alpha=0.5)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

# Imprimir resultados
print("Regresión Lineal Normal:")
print("MSE:", mse_lm)
print("R^2:", r2_lm)
print()

print("Regresión Ridge:")
print("MSE:", mse_ridge)
print("R^2:", r2_ridge)
print()

print("Regresión Lasso:")
print("MSE:", mse_lasso)
print("R^2:", r2_lasso)

print('--------------------------------------------------------------')
print('PROCESAMIENTO FINALIZADO EXITOSAMENTE!!!')
print('--------------------------------------------------------------')

TIEMPO = (time.time() - start_time)/60
print("--- %s Minutos ---" % (TIEMPO))

-------------------INICIO PROCESAMIENTO-----------------
Regresión Lineal Normal:
MSE: 669930.3006431934
R^2: 0.8487875815111954

Regresión Ridge:
MSE: 670876.0610185396
R^2: 0.848574110477672

Regresión Lasso:
MSE: 690713.0200782518
R^2: 0.8440966378928334
--------------------------------------------------------------
PROCESAMIENTO FINALIZADO EXITOSAMENTE!!!
--------------------------------------------------------------
--- 0.6488774617513021 Minutos ---


## Conclusiones

- La carga de datos es un proceso estandar, esto solo cambia dependiendo de la fuente de los datos, la finalidad es la misma, tener un dataframe con información.

- La limpieza y preparación de los datos tomaron aproximadamente el 80% del tiempo para el desarrollo del trabajo, contrastando con un 20% para realizar la parte de Machine Learning.

- El tiempo de procesamiento no fue impedimento aplicando los diferentes algoritmos de Regresión Lineal.

- Los mejores desempeños en la Regresión Lineal Simple se obtuvieron con las variables de entrada "TotalQuantity" y "TotalProductosUnicos".

- En la Regresión Lineal Multiple se consiguieron buenos desempeños aplicando la regresión lineal y sus variantes con regularización Ridge y Lasso, entre estos 3 algoritmos no se encontraron diferencias notorias en los desempeños obtenidos.

